# Import libs and load dfs

In [1]:
# Classic imports
import pandas as pd
import numpy as np

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# - Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Other imports
import tqdm

# Transforming the data

### Attention nouvelle règle : 

- Damien is to come to the next classes with no underwear and a kilt.
- We'll add a value between 0 and 1 showcasing the evolution of the year : 0 for the beginnin of the year, 1 for the end of the year. Then, we'll multiply this by pi and use the sine of this value.
- This way we'll have values closer to 1 for 'colder' days and 0 for 'hotter' days. 

## Sinus of all date data

In [ ]:
def sin_ratios_for_date(modify_df, name):
    # Preprocess the datasets
    # Convert the Sales_Date to datetime format
    modify_df['date'] = pd.to_datetime(modify_df['date'])

    # Extract useful date features (e.g., year, month, day)
    modify_df['Year'] = modify_df['date'].dt.year
    modify_df['Month'] = modify_df['date'].dt.month
    modify_df['Day'] = modify_df['date'].dt.day
    modify_df['sin2_month'] = modify_df['Month'].map(lambda x: ((np.sin(float(x)/12)**2)))  # value will possibly be negative if standardscaler has been applied after calculus of sinus

    modify_df['day_of_year'] = modify_df['date'].dt.dayofyear
    # Calcul du nombre de jours dans l'année (365 ou 366)
    modify_df['days_in_year'] = np.where(modify_df['date'].dt.is_leap_year, 366, 365)
    # Calcul El Famoso ratio (float entre 0 et 1)
    modify_df['day_ratio'] = modify_df['day_of_year'] / modify_df['days_in_year']
    # Calculer le sinus de pi * ratio
    modify_df['sin_dayofyear'] = np.sin(np.pi * modify_df['day_ratio'])
    modify_df.drop(columns=['day_of_year', 'days_in_year', 'day_ratio'], inplace=True)
    modify_df = modify_df.drop(columns=['date', 'Year', 'Month', 'Day'])

    new_name = name + "_with_sin_ratios"

    return modify_df, new_name

## Transform df to have price/square meters

In [3]:
def create_new_df_with_m2(to_modify_df, name, include_price = False):
    # create a new data frame with everything plus a column corresponding to 1 if "m2_jardin" is >0, 0 otherwise
    # we will also do the same for m2_etage and m2_soussol
    modify_df = to_modify_df.copy()
    modify_df['jardin'] = modify_df['m2_jardin'].map(lambda x: 1 if x > 0 else 0)
    modify_df['etage'] = modify_df['m2_etage'].map(lambda x: 1 if x > 0 else 0)
    modify_df['soussol'] = modify_df['m2_soussol'].map(lambda x: 1 if x > 0 else 0)

    # now add three new columns per df, one for interior m2 and one for exterior m2 and one for total m2
    # for interior we already have m2_interieur which is the sum of etage and soussol
    modify_df['m2_outside'] = modify_df['m2_jardin']
    modify_df['m2_total'] = modify_df['m2_interieur'] + modify_df['m2_outside']

    if include_price:
        # now we add two columns per df, one for prix per m2 interior and one for prix per m2 total
        modify_df['prix_m2_interieur'] = modify_df['prix'] / modify_df['m2_interieur']
        modify_df['prix_m2_total'] = modify_df['prix'] / modify_df['m2_total']

    new_name = name + "_with_m2_price"

    return modify_df, new_name

## Scaling the data

In [4]:
def scaling_data(df, name, scaler = None, scaler_type = None):
    if scaler :
        scaler = scaler
    else :
        if scaler_type == 'minmax':
            # Initialize the MinMaxScaler
            scaler = MinMaxScaler()
        elif scaler_type == 'standard':
            # Initialize the StandardScaler
            scaler = StandardScaler()
        else:
            scaler = MinMaxScaler() # Default scaler will be minmax
            #raise ValueError("Invalid scaler type. Please choose between 'minmax' and 'standard'.

    # Scale the data, but not the ID column
    columns_to_scale = df.columns.difference(['id', 'lat', 'long', 'sin_month', 'sin_dayofyear'])
    to_scale_df1 = df[columns_to_scale].copy() # to not modify the original one

    scaled_df1 = df.copy()
    scaled_df1[columns_to_scale] = scaler.fit_transform(to_scale_df1)

    if scaler_type:
        scaled_name = name + "_scaled" + "_" + scaler_type
    else:
        scaled_name = name + "_scaled"

    return scaler, scaled_df1, scaled_name

## Export the csv

In [5]:
def exporting_df(df, name):
    df.to_csv('CSV data/'+ name + '.csv', index=False)


# Call the transfos

In [6]:
# Load the datasets
origin_train_name = "train_with_clusters"
origin_test_name = "test_with_clusters"

train_df = pd.read_csv('CSV DATA/' + origin_train_name + '.csv')  # fichier csv VIERGE d'origine du train, ou clusterisé
test_df = pd.read_csv('CSV DATA/' + origin_test_name + '.csv')  # fichier csv VIERGE d'origine du test, ou clusterisé

#little printy print 
print("Train shape: ", train_df.shape)
print("Test shape: ", test_df.shape)

Train shape:  (17147, 24)
Test shape:  (4287, 23)


In [7]:
# dates to sinus function
train_sin, train_sin_name = sin_ratios_for_date(train_df, origin_train_name)
test_sin, test_sin_name = sin_ratios_for_date(test_df, origin_test_name)

# find price per square meter
train_sin_m2, train_sin_m2_name = create_new_df_with_m2(train_sin, train_sin_name, include_price=True)
test_sin_m2, test_sin_m2_name = create_new_df_with_m2(test_sin, test_sin_name, include_price=False)

# Scale the data
scaler1, train_sin_m2_scaled, train_sin_m2_scaled_name = scaling_data(train_sin_m2, train_sin_m2_name, scaler_type='standard')
scaler1, train_sin_scaled, train_sin_scaled_name = scaling_data(train_sin, train_sin_name, scaler=scaler1, scaler_type='standard')
scaler1, test_sin_m2_scaled, test_sin_m2_scaled_name = scaling_data(test_sin_m2, test_sin_m2_name, scaler=scaler1, scaler_type='standard')
scaler1, test_sin_scaled, test_sin_scaled_name = scaling_data(test_sin, test_sin_name, scaler=scaler1, scaler_type='standard')

In [8]:
train_sin_m2_scaled.head()

,id,prix,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,...,cluster_tres_eleve,sin2_month,sin_dayofyear,jardin,etage,soussol,m2_outside,m2_total,prix_m2_interieur,prix_m2_total
0,4443800785,-0.167369,-1.471744,-1.453014,-0.505115,-0.271088,-1.050087,0.920725,-0.918153,-0.090074,...,-0.500055,1.538983,0.337523,0.0,0.0,1.248548,-0.271088,-0.281097,0.290452,0.600286
1,2722059013,0.018316,-1.471744,-1.453014,-0.885037,0.679263,-0.629499,-0.656952,-0.918153,-0.090074,...,-0.500055,-1.270457,0.296713,0.0,0.0,-0.800930,0.679263,0.657080,1.520718,-1.039685
2,8856004730,-0.923697,-1.471744,0.811236,-0.537680,0.136954,-0.244962,-0.656952,0.004358,-0.090074,...,-0.500055,0.768639,0.785650,0.0,0.0,-0.800930,0.136954,0.124587,-1.256075,-1.113512
3,2473003210,-0.480051,-0.398785,-0.482621,0.254730,-0.175406,-0.208911,0.920725,-0.918153,-0.090074,...,-0.500055,-1.113308,0.580800,0.0,0.0,1.248548,-0.175406,-0.169103,-0.971542,-0.526783
4,6743700335,-0.196971,-0.398785,-0.159157,-0.309726,-0.060588,0.007391,-0.656952,-0.918153,-0.090074,...,-0.500055,-0.326610,0.972118,0.0,0.0,-0.800930,-0.060588,-0.067151,-0.033046,-0.598759


In [9]:
exporting_df(train_sin_m2_scaled, train_sin_m2_scaled_name)
exporting_df(test_sin_m2_scaled, test_sin_m2_scaled_name)

In [10]:
# X_train, y_train, X_test, y_test = train_test_split(train_new.drop(columns=['prix']), train_new['prix'], test_size=0.2, random_state=42)

In [11]:
# clusters = ['cluster_tres_bas', 'cluster_bas', 'cluster_moyen', 'cluster_eleve', 'cluster_tres_eleve']
# train_new.drop(columns=clusters, inplace=True)